# 第3章: 正規表現

Wikipediaの記事を以下のフォーマットで書き出したファイル[jawiki-country.json.gz](http://www.cl.ecei.tohoku.ac.jp/nlp100/data/jawiki-country.json.gz)がある．

- 行に1記事の情報がJSON形式で格納される
- 各行には記事名が"title"キーに，記事本文が"text"キーの辞書オブジェクトに格納され，そのオブジェクトがJSON形式で書き出される
- ファイル全体はgzipで圧縮される

以下の処理を行うプログラムを作成せよ．

## 20. JSONデータの読み込み
Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [73]:
import json
pages = [json.loads(l) for l in list(open('data/jawiki-country.json'))]
open('output/related_uk.json', 'w').write(json.dumps([p for p in pages if 'イギリス' in p['text']]))

14222681

## 21. カテゴリ名を含む行を抽出
記事中でカテゴリ名を宣言している行を抽出せよ．

In [74]:
import json

result = []
for p in json.load(open('output/related_uk.json')):
    for line in p['text'].split('\n'):
        m = re.search('\[\[(Category|カテゴリ):.+\]\]', line)
        if m:
            result.append(line.strip())

print(len(result))
print(result[:3])

625
['[[Category:エジプト|*]]', '[[Category:フランコフォニー]]', '[[Category:共和国]]']


## 22. カテゴリ名の抽出
記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [75]:
import json

result = set()
for p in json.load(open('output/related_uk.json')):
    for m in re.finditer('\[\[(Category|カテゴリ):(.+)\]\]', p['text']):
        result |= set([m.groups()[1].split('|')[0]])

print(len(result))
print(list(result)[:3])

215
['キューバ', '保護国', 'カタール']


## 23. セクション構造
記事中に含まれるセクション名とそのレベル（例えば"== セクション名 =="なら1）を表示せよ．

In [76]:
result = []
for p in json.load(open('output/related_uk.json')):
    for m in re.finditer('(==+) (.+) ==+', p['text']):
        g = m.groups()
        result.append((g[1], len(g[0])))

print(len(result))
result[:3]

5193


[('国号', 2), ('歴史', 2), ('古代エジプト', 3)]

## 24. ファイル参照の抽出
記事から参照されているメディアファイルをすべて抜き出せ．

In [77]:
result = []                                                                                         
for p in json.load(open('output/related_uk.json')):                                                 
    for m in re.finditer('File:(.*?)\||ファイル:(.*?)\|', p['text']):                               
        result += [f for f in m.groups() if f]                                                      
                                                                                                    
print(len(result))                                                                                  
result[:3]

2581


['Coat_of_arms_of_Egypt.svg',
 'All Gizah Pyramids.jpg',
 'Egyptiska hieroglyfer, Nordisk familjebok.png']

## 25. テンプレートの抽出
記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

In [78]:
from collections import defaultdict

result = defaultdict(dict)
for p in json.load(open('output/related_uk.json')):
    for m in re.finditer('{{基礎情報(.*?)}}\n', p['text'], re.MULTILINE|re.DOTALL):
        for m2 in re.finditer('([^|\n]*?)\s*=(.*?)\n', m.groups()[0]):
            g = m2.groups()
            result[p['title']].update({g[0].strip(): g[1].strip()})
            
result['日本']

{'GDP/人': '38,053<ref name="imf-statistics-gdp"/>',
 'GDPMER追記': '',
 'GDP値': '4兆8350億<ref name="imf-statistics-gdp"/>',
 'GDP値MER': '4兆8460億<ref name="imf-statistics-gdp"/>',
 'GDP値元': '名目478兆3,680億<ref name="imf-statistics-gdp">[http://www.imf.org/external/pubs/ft/weo/2012/02/weodata/weorept.aspx?pr.x=73&pr.y=8&sy=2010&ey=2012&scsm=1&ssd=1&sort=country&ds=.&br=1&c=158&s=NGDP%2CNGDPD%2CNGDPDPC%2CPPPGDP%2CPPPPC&grp=0&a= IMF>Data and Statistics>World Economic Outlook Databases>By Countrise>Japan]</ref>',
 'GDP元追記': '',
 'GDP統計年': '2014',
 'GDP統計年MER': '2014',
 'GDP統計年元': '2013',
 'GDP追記': '',
 'GDP順位': '4',
 'GDP順位MER': '3',
 'ISO 3166-1': 'JP / JPN',
 'ISO 3166-1追記': '',
 'ccTLD': '[[.jp]]',
 'ccTLD追記': '',
 '人口値': '1億2653万<ref>[http://esa.un.org/unpd/wpp/Excel-Data/population.htm United Nations Department of Economic and Social Affairs>Population Division>Data>Population>Total Population]</ref>',
 '人口大きさ': '1 E8',
 '人口密度値': '337',
 '人口統計年': '2012',
 '人口追記': '',
 '人口順位': '10',
 '位置画像':

## 26. 強調マークアップの除去
25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ（参考: [マークアップ早見表](https://ja.wikipedia.org/wiki/Help:%E6%97%A9%E8%A6%8B%E8%A1%A8)）．

In [79]:
from collections import defaultdict

result = defaultdict(dict)
for p in json.load(open('output/related_uk.json')):
    for m in re.finditer('{{基礎情報(.*?)}}\n', p['text'], re.MULTILINE|re.DOTALL):
        for m2 in re.finditer('([^|\n]*?)\s*=(.*?)\n', m.groups()[0]):
            g = m2.groups()
            value = re.sub("\|$", "", g[1].strip())
            value = re.sub("'''''(.+)'''''", "\g<1>", value)
            value = re.sub("'''(.+)'''", "\g<1>",  value)
            value = re.sub("''(.+)''", "\g<1>",  value)
            result[p['title']].update({g[0].strip(): value})
            
result['日本']

{'GDP/人': '38,053<ref name="imf-statistics-gdp"/>',
 'GDPMER追記': '',
 'GDP値': '4兆8350億<ref name="imf-statistics-gdp"/>',
 'GDP値MER': '4兆8460億<ref name="imf-statistics-gdp"/>',
 'GDP値元': '名目478兆3,680億<ref name="imf-statistics-gdp">[http://www.imf.org/external/pubs/ft/weo/2012/02/weodata/weorept.aspx?pr.x=73&pr.y=8&sy=2010&ey=2012&scsm=1&ssd=1&sort=country&ds=.&br=1&c=158&s=NGDP%2CNGDPD%2CNGDPDPC%2CPPPGDP%2CPPPPC&grp=0&a= IMF>Data and Statistics>World Economic Outlook Databases>By Countrise>Japan]</ref>',
 'GDP元追記': '',
 'GDP統計年': '2014',
 'GDP統計年MER': '2014',
 'GDP統計年元': '2013',
 'GDP追記': '',
 'GDP順位': '4',
 'GDP順位MER': '3',
 'ISO 3166-1': 'JP / JPN',
 'ISO 3166-1追記': '',
 'ccTLD': '[[.jp]]',
 'ccTLD追記': '',
 '人口値': '1億2653万<ref>[http://esa.un.org/unpd/wpp/Excel-Data/population.htm United Nations Department of Economic and Social Affairs>Population Division>Data>Population>Total Population]</ref>',
 '人口大きさ': '1 E8',
 '人口密度値': '337',
 '人口統計年': '2012',
 '人口追記': '',
 '人口順位': '10',
 '位置画像':

## 27. 内部リンクの除去
26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ（参考: [マークアップ早見表](https://ja.wikipedia.org/wiki/Help:%E6%97%A9%E8%A6%8B%E8%A1%A8)）．

In [80]:
from collections import defaultdict

result = defaultdict(dict)
for p in json.load(open('output/related_uk.json')):
    for m in re.finditer('{{基礎情報(.*?)}}\n', p['text'], re.MULTILINE | re.DOTALL):
        for m2 in re.finditer('([^|\n]*?)\s*=(.*?)\n', m.groups()[0]):
            g = m2.groups()
            value = re.sub("\|$", "", g[1].strip())
            value = re.sub("'''''(.+)'''''", "\g<1>", value)
            value = re.sub("'''(.+)'''", "\g<1>",  value)
            value = re.sub("''(.+)''", "\g<1>",  value)
            for m in re.finditer('\[\[([^[]+)\]\]', value):
                value = re.sub(
                    '\[\[{0}\]\]'.format(re.escape(m.groups()[0])),
                    m.groups()[0].split('|')[-1],
                    value
                )
            result[p['title']].update({g[0].strip(): value})

result['日本']

{'GDP/人': '38,053<ref name="imf-statistics-gdp"/>',
 'GDPMER追記': '',
 'GDP値': '4兆8350億<ref name="imf-statistics-gdp"/>',
 'GDP値MER': '4兆8460億<ref name="imf-statistics-gdp"/>',
 'GDP値元': '名目478兆3,680億<ref name="imf-statistics-gdp">[http://www.imf.org/external/pubs/ft/weo/2012/02/weodata/weorept.aspx?pr.x=73&pr.y=8&sy=2010&ey=2012&scsm=1&ssd=1&sort=country&ds=.&br=1&c=158&s=NGDP%2CNGDPD%2CNGDPDPC%2CPPPGDP%2CPPPPC&grp=0&a= IMF>Data and Statistics>World Economic Outlook Databases>By Countrise>Japan]</ref>',
 'GDP元追記': '',
 'GDP統計年': '2014',
 'GDP統計年MER': '2014',
 'GDP統計年元': '2013',
 'GDP追記': '',
 'GDP順位': '4',
 'GDP順位MER': '3',
 'ISO 3166-1': 'JP / JPN',
 'ISO 3166-1追記': '',
 'ccTLD': '.jp',
 'ccTLD追記': '',
 '人口値': '1億2653万<ref>[http://esa.un.org/unpd/wpp/Excel-Data/population.htm United Nations Department of Economic and Social Affairs>Population Division>Data>Population>Total Population]</ref>',
 '人口大きさ': '1 E8',
 '人口密度値': '337',
 '人口統計年': '2012',
 '人口追記': '',
 '人口順位': '10',
 '位置画像': 'Ja

## 28. MediaWikiマークアップの除去
27の処理に加えて，テンプレートの値からMediaWikiマークアップを可能な限り除去し，国の基本情報を整形せよ．

In [81]:
result = defaultdict(dict)
for p in json.load(open('output/related_uk.json')):
    for m in re.finditer('{{基礎情報(.*?)}}\n', p['text'], re.MULTILINE | re.DOTALL):
        for m2 in re.finditer('([^|\n]*?)\s*=(.*?)\n', m.groups()[0]):
            g = m2.groups()
            value = re.sub("\|$", "", g[1].strip())
            value = re.sub("'''''(.+)'''''", "\g<1>", value)
            value = re.sub("'''(.+)'''", "\g<1>",  value)
            value = re.sub("''(.+)''", "\g<1>",  value)
            # コメントアウトの除去
            value = re.sub("<!--(.+)-->", '', value)
            # 脚注の除去
            value = re.sub("<ref(.+)(\/>|<\/ref>)", '', value)
            for m in re.finditer('\[\[([^[]+)\]\]', value):
                value = re.sub(
                    '\[\[{0}\]\]'.format(re.escape(m.groups()[0])),
                    m.groups()[0].split('|')[-1],
                    value
                )
            # 外部リンクの除去(URLのみ残し)
            for m in re.finditer('\[([^[]+)\]', value):
                value = re.sub(
                    '\[{0}\]'.format(re.escape(m.groups()[0])),
                    m.groups()[0].split(' ')[0],
                    value
                )
            # テンプレートの除去
            for m in re.finditer('({{([^{]+)}})', value):
                value = re.sub(
                    re.escape(m.groups()[0]),
                    m.groups()[1].split('|')[-1],
                    value
                )
            # カテゴリの除去
            for m in re.finditer('\[\[(Category|カテゴリ):([^\[]+)\]\]', value):
                value = re.sub(
                   '\[\[(Category|カテゴリ):{0}\]\]'.format(re.escape(m.groups()[1])),
                    m.groups()[1].split('|')[-1],
                    value
                )
            result[p['title']].update({g[0].strip(): value})

result['日本']

{'GDP/人': '38,053',
 'GDPMER追記': '',
 'GDP値': '4兆8350億',
 'GDP値MER': '4兆8460億',
 'GDP値元': '名目478兆3,680億',
 'GDP元追記': '',
 'GDP統計年': '2014',
 'GDP統計年MER': '2014',
 'GDP統計年元': '2013',
 'GDP追記': '',
 'GDP順位': '4',
 'GDP順位MER': '3',
 'ISO 3166-1': 'JP / JPN',
 'ISO 3166-1追記': '',
 'ccTLD': '.jp',
 'ccTLD追記': '',
 '人口値': '1億2653万',
 '人口大きさ': '1 E8',
 '人口密度値': '337',
 '人口統計年': '2012',
 '人口追記': '',
 '人口順位': '10',
 '位置画像': 'Japan (orthographic projection).svg',
 '元首等氏名': '明仁<br/>（125代目:今上天皇）',
 '元首等肩書': '天皇',
 '公式国名': '',
 '公用語': '日本語（事実上）',
 '国旗画像': 'Flag of Japan.svg',
 '国歌': '君が代File:Kimi ga Yo instrumental.ogg',
 '国歌追記': '',
 '国章リンク': '（国章（慣例上））',
 '国章画像': '十六八重表菊',
 '国際電話番号': '81',
 '国際電話番号追記': '',
 '夏時間': 'なし',
 '建国年月日': '諸説あり（日本神話による初代天皇即位の日をグレゴリオ暦に換算すると前660年2月11日）。',
 '建国形態': '建国',
 '日本語国名': '日本国',
 '時間帯': '+9',
 '時間帯追記': '',
 '最大都市': '東京都区部（23区を一つの自治体と見なす場合。）',
 '標語': 'なし',
 '標語追記': '',
 '水面積率': '0.8%',
 '注記': '',
 '略名': '日本',
 '通貨': '円',
 '通貨コード': 'JPY',
 '通貨追記': '',
 '面積値': '377,961

## 29. 国旗画像のURLを取得する
テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: MediaWiki APIのimageinfoを呼び出して，ファイル参照をURLに変換すればよい）

In [82]:
import json
import re
import requests


def get_country_flag_image(country_name):
    for p in json.load(open('output/related_uk.json')):
        if p['title'] != country_name:
            continue
        for m in re.finditer('{{基礎情報(.*?)}}\n', p['text'], re.MULTILINE | re.DOTALL):
            for m2 in re.finditer('([^|\n]*?)\s*=(.*?)\n', m.groups()[0]):
                g = m2.groups()
                if '国旗画像' not in g[0].strip():
                    continue
                image = re.sub("\|$", '', g[1].strip())
                params = {
                    'action': 'query',
                    'titles': 'File:' + image,
                    'prop': 'imageinfo',
                    'iiprop': 'url',
                    'format': 'json',

                }
                res = requests.get('https://commons.wikimedia.org/w/api.php', params=params)
                pages = res.json()['query']['pages']
                for page_id, data in pages.items():
                    return data['imageinfo'][0]['url']

get_country_flag_image('リヒテンシュタイン')

'https://upload.wikimedia.org/wikipedia/commons/4/47/Flag_of_Liechtenstein.svg'